In this exercise, we want to find urls that matches given regular expression.

D = the content containing lots of urls (maybe big data)
RE = given set of regular expressions

In [71]:
# import libraries
import re
from difflib import SequenceMatcher

In [72]:
def lcs(X, Y, m, n):
    if m == 0 or n == 0: 
       return 0; 
    elif X[m-1] == Y[n-1]: 
       return 1 + lcs(X, Y, m-1, n-1); 
    else: 
       return max(lcs(X, Y, m, n-1), lcs(X, Y, m-1, n)); 

In [77]:
X = "AGGTAB"
Y = "GXTXAYB"
print( "Length of LCS is ", lcs(X , Y, len(X), len(Y)) )

Length of LCS is  4


In [81]:
def RetError(str1, str2):
    #a = lcs(str1, str2, len(str1), len(str2))
    #return a / len(str1)
    s = SequenceMatcher(None, str1, str2)
    return s.ratio()
    

In [82]:
def IsMatch(D, RE, startIdx, endIdx): 
    #print(RE)
    p = re.compile(RE)
    for i in range(startIdx, endIdx):
        #print(D[i])
        x = p.search(D[i])
        if x:
            print(x)
        else:
            print(RetError(D[i], RE))
    return x 

In [83]:
RE = ['https://MyWeb.com/(.*)']

D = ['https://MyWeb.com/1',
     'https://MyWeb.com/2',
     'https://MyWeb.com/3',
     'https://YourWeb.com/1',
     'https://YourWeb.com/2',
     'https://YourWeb.com/3',
    ]

X_arr = IsMatch(D, RE[0], 0, len(D))

p = re.compile(RE[0])
m = p.search(D[0])
print(m)

<_sre.SRE_Match object; span=(0, 19), match='https://MyWeb.com/1'>
<_sre.SRE_Match object; span=(0, 19), match='https://MyWeb.com/2'>
<_sre.SRE_Match object; span=(0, 19), match='https://MyWeb.com/3'>
0.7441860465116279
0.7441860465116279
0.7441860465116279
<_sre.SRE_Match object; span=(0, 19), match='https://MyWeb.com/1'>
